In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.modes import AvirisDataManager
from typing import List, Union, Tuple, Optional, Dict, Callable

ImportError: dlopen(/opt/anaconda3/envs/spectraclass1/lib/python3.8/site-packages/rasterio/_base.cpython-38-darwin.so, 2): Library not loaded: @rpath/libfontconfig.1.dylib
  Referenced from: /opt/anaconda3/envs/spectraclass1/lib/libpoppler.117.0.0.dylib
  Reason: Incompatible library version: libpoppler.117.dylib requires version 14.0.0 or later, but libfontconfig.1.dylib provides version 13.0.0

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [ ]:
dm: DataManager = DataManager.initialize( "img_mgr", 'aviris' )

dm.modal.cache_dir = "/explore/nobackup/projects/ilab/cache1"
dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"
dm.modal.images_glob =  "ang20170731t222832rfl/ang*_rfl_v2p9/ang*_corr_v2p9_img"

block_size = 200
method = "aec" # "vae"
model_dims = 32
version = "v2v2"  # "v2v2" "v2p9"
month = "201908"  # "201707" "201908"
target_block = [1,7]

dm.modal.ext =  "_img"
dm.proc_type = "skl"
dm.modal.images_glob = f"ang{month}*rfl/ang*_rfl_{version}/ang*_corr_{version}_img"
dm.use_model_data = True
dm.preprocess = False
TileManager.block_size = block_size
TileManager.reprocess = False
AvirisDataManager.version = version
TileManager.block_index = target_block
dm.modal.valid_aviris_bands = [ [5,193], [214,283], [319,10000] ]
dm.modal.model_dims = model_dims
dm.modal.reduce_method = method
dm.modal.reduce_nepoch = 3
dm.modal.reduce_target_block = target_block
dm.modal.reduce_focus_nepoch = 10
dm.modal.reduce_niter = 1
dm.modal.reduce_focus_ratio = 10.0
dm.modal.reduce_dropout = 0.0
dm.modal.reduce_learning_rate = 1e-4
dm.modal.refresh_model = False
dm.modal.modelkey = f"b{block_size}.{method}"

Here we load the project data and define a set of class names and associated colors to be used in the classification process.

In [ ]:
dm.loadCurrentProject()
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

lm().setLabels( classes )

Add Custom CNN

In [ ]:
from spectraclass.learn.mlp import MLP
from spectraclass.learn.cnn import CNN2D, SpectralCNN, CNN3D

cm().nepochs = 20
cm().mid = "mlp"

mlp = MLP( 'mlp', layers = [32, 16] )
cm().addNetwork(mlp)

cnn = CNN2D( 'cnn2d', cnn_layers =  [(8,3,1)], dense_layers= [32, 16] )
cm().addNetwork(cnn)

cnn1D = SpectralCNN( 'cnn1d', cnn_layers = [(8,5,3),(8,5,3)], dense_layers= [32, 16] )
cm().addNetwork(cnn1D)

Here we start up the Spectraclass GUI.

In [ ]:
controller: SpectraclassController = app()
controller.gui()